# LJ Tutorial 

Environment setup
```
git clone git@github.com:jennyfothergill/msibi.git
cd msibi
conda env create -f environment.yml
conda activate msibi
pip install -e .
```

In [6]:
import itertools

import freud
import gsd
import gsd.hoomd
import hoomd
import hoomd.md
import matplotlib.pyplot as plt
import mdtraj as md
import numpy as np

from msibi import MSIBI, State, Pair, mie, msibi_context
from msibi.utils.general import get_msibi_instance

Remove old runs: only the rdf targets, run_template scripts, and the start.hoomdxml remain.

In [ ]:
msibi_context

In [ ]:
%%bash
rm -rf state*/* rdfs/pair* potentials/* f_fits.log state*/log.txt state*/err.txt state*/run.py state*/query.dcd

Create the trajectories at the three statepoints... takes about 40 minutes on 2.2 GHz Intel Core i7.

In [ ]:
kTs = [0.5, 1.5, 2.0]
n = 12
n_particles = n**3

In [ ]:
# This cell takes 40 minutes. Don't uncomment and rerun it unless you have a good reason.
# trajectory{kT}.gsd files were created here

#for i,kT in enumerate(kTs):
#    hoomd.context.initialize("")
#    system = hoomd.init.create_lattice(
#        unitcell = (hoomd.lattice.sc(a=1.58, type_name="A")), 
#        n = n,
#    )
#    
#    nl = hoomd.md.nlist.cell()
#    lj = hoomd.md.pair.lj(r_cut=2.5, nlist=nl)
#    lj.pair_coeff.set('A', 'A', epsilon=1.0, sigma=1.0)
#    hoomd.md.integrate.mode_standard(dt=0.001)
#    _all = hoomd.group.all()
#    nvt = hoomd.md.integrate.nvt(group=_all, kT=kT, tau=1)
#    nvt.randomize_velocities(seed=23)
#    hoomd.analyze.log(
#        filename=f'LJ_kT{kT}.log',
#        quantities=["time", "temperature", "potential_energy"],
#        period=100,
#        overwrite=True
#    )
#    hoomd.dump.gsd(f'trajectory{kT}.gsd', period=5e3, group=_all, overwrite=True)
#    hoomd.run(1e6)

Check that the system is equilibrated.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True)
fig.suptitle("Potential Energy vs Timestep")
for i, kT in enumerate(kTs):   
    log = np.loadtxt(f'state{i}/LJ_kT{kT}.log', skiprows=1)
    ax1.plot(log[:,0],log[:,3], label=f"kT={kT}")
    ax2.plot(log[-50:,0],log[-50:,3], label=f"kT={kT}")
    ax1.set_xlabel("Timestep")
    ax2.set_xlabel("Timestep")
    ax1.set_ylabel("Potential Energy (AU)")
    ax1.legend()
    ax2.legend()

Last 50 frames look equilibrated (TODO autocorrelation?), so we'll calculate the RDF and use those as our targets.

In [ ]:
for i,kT in enumerate(kTs):
    gsdfile = f'state{i}/trajectory{kT}.gsd'
    with gsd.hoomd.open(gsdfile) as t:
        rdf = freud.density.RDF(bins=101, r_max=5)
        for snap in t[-50:]:
            rdf.compute(system=snap, reset=False)
        data = np.stack((rdf.bin_centers, rdf.rdf)).T
        np.savetxt(f"rdfs/rdftarget{i}", data)
        plt.plot(rdf.bin_centers, rdf.rdf)
        plt.title(f"State {i} kT = {kT}")
        plt.xlabel("r")
        plt.ylabel("g(r)")
        plt.show()

Set up global parameters:

- rdf_cutoff specifies how far out to calculate the rdf
- pot_cutoff specifies where to cut off the IBI calculation of the potential. The tail of the potential function (by default the last 5 points will be smoothed to zero using the [XPLOR smoothing function](https://hoomd-blue.readthedocs.io/en/stable/module-md-pair.html#hoomd.md.pair.pair). Small r values of the potential are also corrected using a linear correction by default. 
- smooth_rdfs if True, applies a Savitzky-Golay filter to the rdf array

In [8]:
rdf_cutoff = 5.0
opt = MSIBI(
    rdf_cutoff=rdf_cutoff, n_rdf_points=101, pot_cutoff=3.0, smooth_rdfs=True, verbose=True
)

_msibi = get_msibi_instance()
print(_msibi.rdf_cutoff)
print(_msibi.n_rdf_points)

5.0
101


In [ ]:
msibi.rdf_cutoff

In [ ]:
import gc
for obj in gc.get_objects():
    if isinstance(obj, msibi.optimize.MSIBI):
        _MSIBI = obj

In [ ]:
_MSIBI.rdf_cutoff

In [ ]:
_MSIBI.n_rdf_points

## Specify states

In [ ]:
# New way
stateA = State(name="A", kT=0.5, traj_file="trajectory0.5.gsd", alpha=0.25)
stateB = State(name="B", kT=1.5, traj_file="trajectory1.5.gsd", alpha=0.50)
stateC = State(name="C", kT=2.0, traj_file="trajectory2.0.gsd", alpha=0.25)

In [ ]:
# Old Way

state0 = State(
    kT=0.5, 
    state_dir='./state0', 
    traj_file='trajectory0.5.gsd',
    name='state0', 
    backup_trajectory=True
)
state1 = State(
    kT=1.5, 
    state_dir='./state1', 
    traj_file='trajectory1.5.gsd',
    name='state1',
    backup_trajectory=True
)
state2 = State(
    kT=2.0, 
    state_dir='./state2', 
    traj_file='trajectory2.0.gsd',
    name='state2', 
    backup_trajectory=True
)
states = [state0, state1, state2]

## Specify pairs

In [ ]:
indices = list(itertools.combinations(range(n_particles), 2))  
initial_guess = mie(opt.pot_r, 1.0, 1.0)
pair0 = Pair(type1="A", type2="A", potential=initial_guess)

In [ ]:
pair0.add_state(stateA, calculate_target_rdf=True)
pair0.add_state(stateB, calculate_target_rdf=True)
pair0.add_state(stateC, calculate_target_rdf=True)

In [ ]:
# all-all for n_particles atoms
# TODO use freud neighborlist?
# TODO write some logic to automate indices generation
indices = list(itertools.combinations(range(n_particles), 2))  

# 1-D array of potential values.
#TODO shouldn't this be boltzmann inverse?
initial_guess = mie(opt.pot_r, 1.0, 1.0)  # np array shape (61,)
rdf_targets = [
    np.loadtxt(f"rdfs/rdftarget{i}") for i in range(3)
]

pair0 = Pair('A', 'A', initial_guess)
alphas = [1.0, 1.0, 1.0]

Add targets to pair

In [ ]:
for state, target, alpha in zip(states, rdf_targets, alphas):
    pair0.add_state(state, target, alpha, indices)
pairs = [pair0]  # optimize() expects a list of pairs

Implement MSIBI algorithm

In [ ]:
opt.optimize(states, pairs, n_iterations=5, engine='hoomd')

Plot results

In [ ]:
for state in states:
    plt.title(f'{state.name} kt={state.kT}')
    
    for step in range(0,opt.n_iterations):
        try:
            step_rdf = np.loadtxt(f'rdfs/pair_A-A-state_{state.name}-step{step}.txt')
            plt.plot(step_rdf[:,0],step_rdf[:,1], label=f'step {step}')
        except OSError:
            break
    target = np.loadtxt(f'rdfs/rdf.target{state.name.strip("state")}.t1t1.txt')
    plt.plot(target[:,0], target[:,1], label='target')
    plt.legend()
    plt.show()

In [ ]:
for step in range(0, opt.n_iterations):
    plt.ylim([-1.5,0.5])
    plt.xlim([0.5,2])
    plt.title("MSIBI potentials by iteration")
    try:
        step_pot = np.loadtxt(f'potentials/step{step}.pot.A-A.txt')
        plt.plot(step_pot[:,0],step_pot[:,1], label=f'step {step}')
    except OSError:
        break
plt.legend()
plt.show()